# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un dataset con reviews de películas.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

df= pd.read_csv ("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df 

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
def hasgood (str_):
#Función devuelve True si el string contiene la palabra good, caso contrario devuelve False
    return 'good' in str_ 

# Revisa cada columna del "review" del dataset
reviewHasGood = df['review'].apply(hasgood) 
reviewHasGood.value_counts()

review
False    31229
True     18771
Name: count, dtype: int64

In [19]:
# Buscar cuantas reviews contienen la frase "not bad" - se la cambia en la varialbe y no se agrega nada a la funcion
def hasquery(str_, query):
    return query in str_

query = 'not bad'
reviewHasAnyQuery = df['review'].apply(lambda x: hasquery(x, query))  

reviewHasAnyQuery.value_counts()



review
False    49752
True       248
Name: count, dtype: int64

In [23]:
# Accede a una review específica usando su indice
df['review'].loc[14]

"This a fantastic movie of three prisoners who become famous. One of the actors is george clooney and I'm not a fan but this roll is not bad. Another good thing about the movie is the soundtrack (The man of constant sorrow). I recommand this movie to everybody. Greetings Bart"

## Funcion que busque una palabra ingresada por el usuario
Busca una palabra en todos los documentos/reviews y muestra en qué documentos aparece

In [28]:
def buscar_palabra_en_documentos(palabra):
   
    print(f"Buscando la palabra: '{palabra}'")
    print("-------------------------------------" )
    
    documentos_encontrados = []
    
    # Revisar cada review
    for indice, review in enumerate(df['review']):
        if palabra.lower() in review.lower():
            documentos_encontrados.append(indice)
    
   # Mostrar resultados
    if documentos_encontrados:
        print(f"La palabra '{palabra}' aparece en {len(documentos_encontrados)} documentos")
    else:
        print(f"La palabra '{palabra}' no aparece en ningún documento")
    return documentos_encontrados

# Variable quemada a buscar
palabra_buscar = "excellent"
resultados = buscar_palabra_en_documentos(palabra_buscar)

Buscando la palabra: 'excellent'
-------------------------------------
La palabra 'excellent' aparece en 3625 documentos


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [29]:
inverted_index = {}

# Procesar solo una parte del dataset 
subset = df.head(5000)

for i, review in enumerate(subset['review']):
    words = review.split()
    for word in set(words):  # evita duplicar índices por palabra
        inverted_index.setdefault(word, []).append(i)
print("Tamanio del índice invertido:", len(inverted_index))


Tamanio del índice invertido: 101552


In [30]:
def search_in_index(query, index, df):
    if query in index:
        doc_ids = index[query]
        print(f"La palabra '{query}' aparece en {len(doc_ids)} documentos.")
        return df.iloc[doc_ids]
    else:
        print(f"La palabra '{query}' no aparece en ningún documento.")
        return pd.DataFrame()

resultados = search_in_index('good', inverted_index, subset)
resultados.head()

## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño.
      -  Corpus grande.
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

In [36]:
import time
import pandas as pd

# Creamos corpus pequenio y grande
corpus_small = df['review'].head(2000).tolist()
corpus_large = df['review'].head(20000).tolist()

# Funcion Bsqueda lineal
def linear_search(query, corpus):
    result = []
    for i, text in enumerate(corpus):
        if query in text:
            result.append(i)
    return result

# Funcion construir indice invertido
def build_inverted_index(corpus):
    inverted = {}
    for i, text in enumerate(corpus):
        words = text.split()
        for word in set(words):
            inverted.setdefault(word, []).append(i)
    return inverted

# Funcion de busqueda usando indice
def index_search(query, index):
    return index.get(query, [])


In [33]:
# Construccion indices
start = time.time()
index_small = build_inverted_index(corpus_small)
time_small_index = time.time() - start

start = time.time()
index_large = build_inverted_index(corpus_large)
time_large_index = time.time() - start

print("Tiempo construccion índice – Corpus pequenio:", time_small_index)
print("Tiempo construccion índice – Corpus grande:", time_large_index)


Tiempo construcción índice – Corpus pequeño: 0.12382030487060547
Tiempo construcción índice – Corpus grande: 1.8273990154266357
